In [14]:
import numpy as np
import pandas as pd

In [15]:
class TwoBodyDecayGenerator:
    """
    Generator for a particle decaying into two identical daughter particles.
    
    The parent particle travels in the +z direction in the lab frame.
    Daughter particles are sampled uniformly in solid angle in the CM frame.
    """
    
    # PDG masses in GeV/c^2
    MASSES = {
        'pion': 0.13957,      # charged pion
        'kaon': 0.49368,      # charged kaon
    }
    
    def __init__(self, n_samples, parent_energy, parent_mass, daughter_type):
        """
        Initialize the decay generator.
        
        Parameters:
        -----------
        n_samples : int
            Number of decay events to generate
        parent_energy : float
            Energy of parent particle in lab frame (GeV)
        parent_mass : float
            Mass of parent particle (GeV/c^2)
        daughter_type : str
            Type of daughter particles ('pion' or 'kaon')
        """
        self.n_samples = n_samples
        self.parent_energy = parent_energy
        self.parent_mass = parent_mass
        self.daughter_type = daughter_type.lower()
        
        if self.daughter_type not in self.MASSES:
            raise ValueError(f"Unknown daughter type: {daughter_type}. Use 'pion' or 'kaon'")
        
        self.daughter_mass = self.MASSES[self.daughter_type]
        
        # Check if decay is kinematically allowed
        if parent_mass < 2 * self.daughter_mass:
            raise ValueError(
                f"Decay not allowed: parent mass ({parent_mass} GeV) < "
                f"2 × daughter mass ({2*self.daughter_mass} GeV)"
            )
        
        # Calculate parent momentum in lab frame
        self.parent_momentum = np.sqrt(parent_energy**2 - parent_mass**2)
        
        # Calculate boost parameters (parent rest frame -> lab frame)
        self.gamma = parent_energy / parent_mass
        self.beta = self.parent_momentum / parent_energy
        
    def generate(self):
        """
        Generate decay events.
        
        Returns:
        --------
        pd.DataFrame with columns:
            - 'daughter1_4momentum': numpy array [E, px, py, pz] for first daughter
            - 'daughter2_4momentum': numpy array [E, px, py, pz] for second daughter
        """
        # Energy of each daughter in CM frame (equal for identical particles)
        E_cm = self.parent_mass / 2
        
        # Momentum magnitude of each daughter in CM frame
        p_cm = np.sqrt(E_cm**2 - self.daughter_mass**2)
        
        # Sample angles uniformly in solid angle
        cos_theta = np.random.uniform(-1, 1, self.n_samples)
        phi = np.random.uniform(0, 2*np.pi, self.n_samples)
        
        sin_theta = np.sqrt(1 - cos_theta**2)
        
        # Daughter 1 momentum in CM frame (arbitrary direction)
        p1_cm_x = p_cm * sin_theta * np.cos(phi)
        p1_cm_y = p_cm * sin_theta * np.sin(phi)
        p1_cm_z = p_cm * cos_theta
        
        # Daughter 2 momentum in CM frame (opposite direction)
        p2_cm_x = -p1_cm_x
        p2_cm_y = -p1_cm_y
        p2_cm_z = -p1_cm_z
        
        # Boost to lab frame (parent moving in +z direction)
        # Lorentz boost only affects z-component and energy
        p1_lab_x = p1_cm_x
        p1_lab_y = p1_cm_y
        p1_lab_z = self.gamma * (p1_cm_z + self.beta * E_cm)
        E1_lab = self.gamma * (E_cm + self.beta * p1_cm_z)
        
        p2_lab_x = p2_cm_x
        p2_lab_y = p2_cm_y
        p2_lab_z = self.gamma * (p2_cm_z + self.beta * E_cm)
        E2_lab = self.gamma * (E_cm + self.beta * p2_cm_z)
        
        # Create 4-momentum arrays [E, px, py, pz]
        daughter1_4momenta = np.column_stack([E1_lab, p1_lab_x, p1_lab_y, p1_lab_z])
        daughter2_4momenta = np.column_stack([E2_lab, p2_lab_x, p2_lab_y, p2_lab_z])
        
        # Create DataFrame
        df = pd.DataFrame({
            'daughter1_4momentum': list(daughter1_4momenta),
            'daughter2_4momentum': list(daughter2_4momenta)
        })
        
        return df
    
def calculate_invariant_mass(events_df):
    """
    Calculate the invariant mass of the two-daughter system for each event.
    
    Parameters:
    -----------
    events_df : pd.DataFrame
        DataFrame with 'daughter1_4momentum' and 'daughter2_4momentum' columns,
        where each entry is a numpy array [E, px, py, pz]
    
    Returns:
    --------
    np.ndarray
        Array of invariant masses (GeV/c^2) for each event
    """
    # Extract 4-momenta as numpy arrays
    d1 = np.array(events_df['daughter1_4momentum'].tolist())
    d2 = np.array(events_df['daughter2_4momentum'].tolist())
    
    # Sum the 4-momenta
    total_4momentum = d1 + d2
    
    # Extract components
    E_total = total_4momentum[:, 0]
    px_total = total_4momentum[:, 1]
    py_total = total_4momentum[:, 2]
    pz_total = total_4momentum[:, 3]
    
    # Calculate invariant mass: m^2 = E^2 - p^2
    p_squared = px_total**2 + py_total**2 + pz_total**2
    m_squared = E_total**2 - p_squared
    
    # Take square root (should always be positive for physical events)
    invariant_mass = np.sqrt(np.abs(m_squared))
    
    return invariant_mass



In [18]:
# Generate 1000 events of a 5 GeV, 3 GeV/c^2 particle decaying to pions
generator = TwoBodyDecayGenerator(
    n_samples=1600000,
    parent_energy=4.3,      # GeV
    parent_mass=1.05,        # GeV/c^2
    daughter_type='pion'      # 'pion' or 'kaon'
)
    
events = generator.generate()
inv_masses = calculate_invariant_mass(events)

In [19]:
def write_fake_lhe_files(df, output_dir, output_name, beam_energy = 3.74, daughter_type= 'kaon', events_per_file = 10000):

    if daughter_type.lower() == 'kaon':
        m = 0.49368  # GeV/c^2
        pdg_id = 321
    elif daughter_type.lower() == 'pion':
        m = 0.13957  # GeV/c^2
        pdg_id = 211

    # Split the DataFrame into smaller DataFrames for writing to LHE files
    num_dfs = int(np.ceil(len(df) / events_per_file))
    dfs = []

    for my_index in np.arange(0, num_dfs, 1):
        df_i = df.iloc[my_index*events_per_file:(my_index+1)*events_per_file].reset_index(drop=True)

        # Create lhe file
        file = open(output_dir+'/'+output_name+"_"+str(my_index)+".lhe", "w")

        # Constants for the LHE file
        rscale = 1.740000e+02 
        alpha_EM = 7.297353e-03
        alpha_QCD = 1.078706E-01 

        #Write header 
        file.write(f"\n")
        file.write(f"<LesHouchesEvents version=\"3.0\">\n")
        file.write(f"<header>\n")
        file.write(f"#  Number of Events        :       "+str(len(df_i))+"\n")
        file.write(f"</header>\n")

        # Write the init block
        file.write(f"<init>\n")
        file.write(f"{11} {623} {beam_energy:.6e} {rscale:.6e} {0} {0} {0} {0} {0} {1}\n")
        file.write("0 0 0 1\n")
        file.write(f"</init>\n")


        # Loop through the DataFrame to write the event blocks
        for index, row in df_i.iterrows():

            # Start event block
            file.write(f"<event>\n")                    
            file.write(f"{2:>2}{1:>7} {0} {rscale:.8e} {alpha_EM:.8e} {alpha_QCD:.8e} \n")

            p1 = row.daughter1_4momentum
            p2 = row.daughter2_4momentum

            file.write(f"{pdg_id:>9}{1:>3}{0:>5}{0:>5}{0:>5}{0:>5}{p1[1]:>+18.10e}{p1[2]:>+18.10e}{p1[3]:>+18.10e}{ p1[0]:>17.10e}{m:>17.10e}{0:>11.4e} {0:>.4e}\n")
            file.write(f"{-pdg_id:>9}{1:>3}{0:>5}{0:>5}{0:>5}{0:>5}{p2[1]:>+18.10e}{p2[2]:>+18.10e}{p2[3]:>+18.10e}{ p2[0] :>17.10e}{m:>17.10e}{0:>11.4e} {0:>.4e}\n")

            # end event block
            file.write(f"</event>\n")

        file.write(f"</LesHouchesEvents>")
        file.close()
        print("LHE written to: ", output_dir+'/'+output_name+"_"+str(my_index)+".lhe")


In [20]:
# outpute LHE file name
output_name = "1pt05"

# output directory
output_dir = "/Users/mghrear/Desktop/fake_lhe/"

write_fake_lhe_files(events, output_dir, output_name, events_per_file = 10000)

LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_0.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_1.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_2.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_3.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_4.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_5.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_6.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_7.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_8.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_9.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_10.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_11.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_12.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_13.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_14.lhe
LHE written to:  /Users/mghrear/Desktop/fake_lhe//1pt05_15.lhe
LH